In [8]:
import pandas as pd
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils
url="444.csv"
dataset = pd.read_csv(url, sep='|', encoding='UTF-8')
dataset_len = len(dataset)

###############################################################################################################################
According the structure of dataset
Because of the trouble with finding a context of hyponyms in some cases (which was discussed on previous Wed, I added one more row 'hyponym2'. If it was possible to find a direct context hyponym (from the task) and hyperonym, the values of rows are the same. But in case of inability to get this pair it was replaced by hyperonim. (Smthg like Plenty in Plenty)
##############################################################################################################################

In [9]:
print(dataset.head(5))

         hyponim       hyponim1                       hyperonim  \
0  Капельмейстер  Капельмейстер    руководитель хоровой капеллы   
1  Капельмейстер  Капельмейстер                        дирижер    
2  Капельмейстер  Капельмейстер  руководитель военного оркестра   
3  Капельмейстер        дирижер                      профессия    
4  Капельмейстер      Профессия      род трудовой деятельности    

                                                text  \
0       Капельмейстер - руководитель хоровой капеллы   
1   Капельмейстер - дирижер военного духового орк...   
2     Капельмейстер - руководитель военного оркестра   
3   Дирижер — профессия штучная, эксклюзивная, ко...   
4   Профессия  -  род трудовой деятельности челов...   

                                                link  bool  \
0                                                  1   0.0   
1  https://kartaslov.ru/%D0%B7%D0%BD%D0%B0%D1%87%...   1.0   
2  https://kartaslov.ru/%D0%B7%D0%BD%D0%B0%D1%87%...   1.0   
3           

In [10]:
START = or_(rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN')),
        rule(gram('ADVB'))
    ).optional(),
    gram('NOUN'),
    or_(
        rule(gram('ADJF')).repeatable(),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')  
    ),
    rule(gram('NOUN')))

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),
    or_(
        eq('принадлежит')
    )
)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также'),
    eq('весьма')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    ),
    rule(
        eq('-')
    ),
    rule(
        eq('–'),
        eq('это'),
        eq('не')
    ),
    rule(
    eq('является')
    ),
    rule(
        eq('-'),
        eq('это')
    ),
    rule(
    or_
        (eq('называют'),
        eq('называется'),
        eq('называли'))
    ),
    rule(
    eq('считается')
    ),
    rule(eq('то'),
        eq('есть')),
    rule(
    eq(','),
    eq('как')
    ),
    rule(
    eq('один'),
    eq('из')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    ), 
    rule(
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADVB'),
        gram('ADJF'),
        gram('NOUN')
    )
)


Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = or_(rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
        ))

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)



In [11]:
import pymorphy2 
morph = pymorphy2.MorphAnalyzer()
def plur(word):
    try:
        b = morph.parse(word)[0]
        c = b.inflect({'plur'}) 
        return (c.word)
    except: 
        return (word)

This part of code is faster because it compare one value of hyponym with one value of text correspondingly 

In [12]:
import numpy as np
def get_hyperonyms_rw1(i): 
    word = dataset['hyponim1'][i]
    array_of_main_words = []
    array_of_main_words.append(word.lower())
    if (plur(word) != word.lower()):
        array_of_main_words.append(plur(word))
    array_of_hyper = []
    for main_word in array_of_main_words:    
        HYPONYM = eq(utils.deaccent(main_word))
        RULE = or_(
            rule(HYPONYM, ATAKJE, START, MID, END),
            rule(HYPONYM, MID, END),
            rule(START_S, END, KAK, HYPONYM),
            rule(END, INCLUDING, HYPONYM),
            #rule(HYPONYM, MID, RULE1),
            rule(HYPONYM, MID, END, END),
            rule(HYPONYM, MID, START, END),
            rule(START, MID, HYPONYM),
            rule(HYPONYM, END)
            #rule(HYPONYM, MID, RULE1, END)
            )
        parser = Parser(RULE) 
        text = utils.deaccent(dataset['text'][i])
        text = re.sub(r'\(.+?\)', '', text)
        text = text.lower().replace('* сергии радонежскии* ', '')
        for idx, match in enumerate(parser.findall(text.lower())):
            k = [_.value for _ in match.tokens]
            array_of_hyper.append(k)
    if (len(array_of_hyper) > 0):
        print(array_of_hyper)


This code replace 'й' to 'и', and sometimes it changes part of speach. Example is given below. Because of this sometimes rule adj + noun doesn't work 

In [13]:
print(morph.parse('хоровои'))
print(morph.parse('хоровой'))


[Parse(word='хоровои', tag=OpencorporaTag('NOUN,inan,masc plur,nomn'), normal_form='хоровой', score=0.5, methods_stack=((<DictionaryAnalyzer>, 'вои', 175, 6), (<UnknownPrefixAnalyzer>, 'хоро'))), Parse(word='хоровои', tag=OpencorporaTag('NOUN,inan,masc plur,accs'), normal_form='хоровой', score=0.5, methods_stack=((<DictionaryAnalyzer>, 'вои', 175, 9), (<UnknownPrefixAnalyzer>, 'хоро')))]
[Parse(word='хоровой', tag=OpencorporaTag('ADJF masc,sing,nomn'), normal_form='хоровой', score=0.16666666666666666, methods_stack=((<DictionaryAnalyzer>, 'хоровой', 136, 0),)), Parse(word='хоровой', tag=OpencorporaTag('ADJF inan,masc,sing,accs'), normal_form='хоровой', score=0.16666666666666666, methods_stack=((<DictionaryAnalyzer>, 'хоровой', 136, 4),)), Parse(word='хоровой', tag=OpencorporaTag('ADJF femn,sing,gent'), normal_form='хоровой', score=0.16666666666666666, methods_stack=((<DictionaryAnalyzer>, 'хоровой', 136, 8),)), Parse(word='хоровой', tag=OpencorporaTag('ADJF femn,sing,datv'), normal_for

In [14]:
for i in range(len(dataset)):
    get_hyperonyms_rw1(i)
    if(dataset['bool'][i] == 0):
        print('this word was not find in context as a pair "hyponym + hyperonim", there is a reason why')
        print(dataset['comment'][i])
    

[['капельмеистер', '-', 'руководитель', 'хоровои', 'капеллы']]
this word was not find in context as a pair "hyponym + hyperonim", there is a reason why
nan
[['капельмеистер', '-', 'дирижер', 'военного', 'духового', 'оркестра']]
[['капельмеистер', '-', 'руководитель', 'военного', 'оркестра']]
[['дирижер', '—', 'профессия']]
[['профессия', '-', 'род', 'трудовои', 'деятельности', 'человека']]
this word was not find in context as a pair "hyponym + hyperonim", there is a reason why
капельмейстер=>дирижер=>профессия
[['каперсы', '-', 'род', 'растении', 'семеиства']]
[['каперсы', '—', 'нераспустившиеся', 'бутоны', 'колючего', 'кустарника']]
[['каперсы', '–', 'это', 'не', 'еда']]
[['каперс', '-', 'забытыи', 'овощ'], ['присутствие', 'такои', 'пряности', ',', 'как', 'каперсы']]
[['каперс', '-', 'ароматическии', 'и', 'возбуждающии', 'плод', 'кустарника']]
[['каприс', 'называют', 'пьесы']]
[['пьеса', '-', 'видовое', 'название', 'произведении', 'драматургии']]
this word was not find in context as a

каустик => каустическая сода => неорганическое хим вещество
[['каустик', '—', 'техническое', 'название', 'едких', 'щелочеи']]
[['каустик', '-', 'окись', 'натрия']]
[['оксид', '-', 'окись']]
this word was not find in context as a pair "hyponym + hyperonim", there is a reason why
синонимы
[['кашпо', ',', 'как', 'предмет', 'декора']]
[['кашпо', '—', 'декоративныи', 'сосуд']]
[['кашпо', '-', 'декоративная', 'ваза']]
[['кашпо', '-', 'крашения']]
[['кашпо', '-', 'вместилище', 'из', 'керамики']]
[['каякинг', '-', 'один', 'из', 'видов', 'водного', 'туризма']]
[['каякинг', '-', 'это', 'вид', 'экстримального', 'спорта']]
[['каякинг', '-', 'это', 'путешествия']]
[['каякинг', '-', 'разновидность', 'возного', 'туризма']]
[['каякинг', '-', 'это', 'движение', 'в', 'бурном', 'потоке', 'воды']]
